This notebook provides an example for how to run the Julia version of VSDM. This will walk through calculating the $f_{n \ell m}$ coefficients for both $g_\chi$ and $f_s^2$, all the way to calculating the rate using the example functions from [arXiv:2310.01483](https://arxiv.org/abs/2310.01483)

# Preamble:

In [1]:
using VSDM

# Useful constant:
ckms = 1/VSDM.km_s
;

# $g_\chi$ Coefficients

We begin by defining a wavelet basis `Wavelet(umax)`, whose only parameter is $u_{\textrm{max}}$. We must use natural units for gx so that it is compatible with $f_s^2$ later on.

In [2]:
gx_wavelet = Wavelet(960.0/ckms)

Wavelet(0.0032022202060095)

`VSDM.jl` includes a `GaussianF` struct, which defines a Gaussian based on the parameters `GaussianF(c, mu_vec, sigma)`

$$
g(\vec{x}) = \frac{c}{\pi^{3/2} \sigma^3} \exp(-\frac{|\vec{x}-\vec{\mu}|^2}{\sigma^2})
$$

Note that we expect $\vec{\mu} = (r, \theta, \phi)$ in spherical coordinates

We will now define the gaussians from eqn. 4.3 of 2310.01483 in this way, utilizing the convenience function `VSDM.cart_to_sph` to convert our cartesian vectors to spherical coordinates. Defining a `Vector{GaussianF}` makes it convenient to calculate the coefficients later on.

In [3]:
g0 = GaussianF(0.4, VSDM.cart_to_sph([0.0, 0.0, -230.0/ckms]), 220.0/ckms)

g1 = GaussianF(0.3, VSDM.cart_to_sph([80.0/ckms, 0.0, -80.0/ckms]), 70.0/ckms)

g2 = GaussianF(0.2, VSDM.cart_to_sph([-120.0/ckms, -250.0/ckms, -150.0/ckms]), 
               50.0/ckms)

g3 = GaussianF(0.1, VSDM.cart_to_sph([50.0/ckms, 30.0/ckms, -400.0/ckms]), 
               25.0/ckms)

gaussians_vector = [g0, g1, g2, g3]

4-element Vector{GaussianF}:
 GaussianF(0.4, [0.0007671985910231093, 3.141592653589793, 0.0], 0.0007338421305438437)
 GaussianF(0.3, [0.0003773852704203168, 2.356194490192345, 0.0], 0.00023349522335485936)
 GaussianF(0.2, [0.0010516546652635458, 2.066636841208666, 4.26486900522752], 0.00016678230239632811)
 GaussianF(0.1, [0.001348360391965612, 2.9968384520031814, 0.5404195002705842], 8.339115119816406e-5)

To calculate the projection into the Haar wavelet / spherical harmonic basis, use the function `ProjectF(f, (n_max, ell_max), basis)` which automatically knows how to handle a `Vector{GaussianF}`

Generally, `n_max` should be $2^\lambda - 1$ where $\lambda$ is the number of generations

In [5]:
proj_gx = ProjectF(gaussians_vector, (2^7-1, 4), gx_wavelet)

ProjectedF{Float64, Wavelet}([1.4879937444888378e7 -3.845019507712558e6 … -6.501890473868838e6 -293846.29300014715; 3.706154173580447e7 -1.0172941604187045e7 … -1.720234503073054e7 -777438.3125042468; … ; 16.229784132990204 -2.82828352904991e-60 … -3.20619857922601e-53 1.1335621827125772e-53; 10.628544928338687 -1.585716564658631e-63 … -2.047346515710023e-55 7.238462913591756e-56], [(0, 0), (1, -1), (1, 0), (1, 1), (2, -2), (2, -1), (2, 0), (2, 1), (2, 2), (3, -3)  …  (3, 3), (4, -4), (4, -3), (4, -2), (4, -1), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)], Wavelet(0.0032022202060095))

This returns a `ProjectedF` object, which stores

`proj_gx.fnlm` : The $n \ell m$ coefficients organized in a matrix of $n \times (\ell,m)$

`proj_gx.lm` : The list of corresponding $(\ell, m)$ values

`proj_gx.radial_basis` : stores `gx_wavelet` for further functions

# $f_s^2$ Coefficients

We start by defining $f_s^2$ from eqn 4.8 of 2310.01483 in spherical coordinates:

In [6]:
function fj2(nj, qLj)
    if qLj == 0.0
        if nj == 1
            return 1.0
        else
            return 0.0
        end
    end
    qlp = abs(qLj)/π
    t1 = sinc(0.5*(qlp - nj + 1.0)) / (1 + (nj-1)/qlp)
    t2 = sinc(0.5*(qlp - nj - 1.0)) / (1 + (nj+1)/qlp)
    return (t1+t2)^2
end

a0 = 1 / (VSDM.qBohr)
Length = [4.0, 7.0, 10.0] .* a0

fs2(Lvec, nz, q_xyz) = prod(fj2.([1, 1, nz], Lvec.*q_xyz))
fs2_model(qSph) = fs2(Length, 2, VSDM.sph_to_cart(qSph))

fs2_model (generic function with 1 method)

In order to increase the efficiency of the integrator, it is helpful to explicitly define some symmetries of our function. To do so, we create a `f_uSph` object, where

```
    f_uSph(f::Function; z_even=false, phi_even=false, 
        phi_cyclic=1, phi_symmetric=false)

Struct that adds decorations to a function f(u, θ, φ) that tell ProjectF 
various properties about the function to speed up integration.

z_even: (boolean) if f_uSph(x,y,z) = f_uSph(x,y,-z)
            implies <lm|f> = 0 if (l+m) is odd
phi_even: (boolean) if f_uSph(u,theta,phi) = f_uSph(u,theta,-phi)
            implies <lm|f> = 0 if m is odd
phi_cyclic: (integer) if f_uSph(u,theta,phi) = f_uSph(u,theta,phi + 2*pi/n)
phi_symmetric: (boolean) if f_uSph(u,theta) independent of phi
```

In this case, we have `z_even=true, phi_even=true, phi_cyclic=2`

In [7]:
fs2_uSph = f_uSph(fs2_model; z_even=true, phi_even=true, phi_cyclic=2)

f_uSph(fs2_model, true, true, 2, false)

Then we go ahead and define the wavelet basis and run ProjectF. This will take a bit longer than for the gaussians because we now have to do 3D integrals instead of 1D.

In [8]:
fs2_wavelet = Wavelet(10*VSDM.qBohr)
proj_fs2 = ProjectF(fs2_uSph, (2^7-1,4), fs2_wavelet)

ProjectedF{Float64, Wavelet}([0.0009739106439870224 0.00023089315460593922 … 0.00012722252971116433 0.00010029337347640467; 0.0025765367778145852 0.0006110568065086128 … 0.00033683116173521176 0.0002649937876958776; … ; -1.2388699805667269e-10 1.361157628385514e-10 … 4.4779088820235744e-10 -2.5038308722440645e-10; -1.9434609759610225e-10 1.9496033028964746e-10 … 2.2431345163994087e-10 8.814826538739453e-10], [(0, 0), (2, 0), (2, 2), (4, 0), (4, 2), (4, 4)], Wavelet(37289.47137978341))

# Calculating the Rate

To begin, we define the model parameters in a `ModelDMSM` struct:

```
    ModelDMSM(fdm_n, mX, mSM, deltaE)

Stores the relevant model parameters:

fdm_n : The power of the dark matter form factor F_DM(q) = (α m_e / q)^fdm_n
mX : dark matter mass in eV
mSM : mass of target particle in eV
deltaE : discrete final state energy in eV
```

Note that `VSDM.jl` only supports discrete final state energies at this time.

In [9]:
fdmn = 0
mX = 100e6 # eV
mSM = 511e3 # eV, mass of electron
deltaE = 4.03 # eV, energy of the first excited state for our fs2

model = ModelDMSM(fdmn, mX, mSM, deltaE)

ModelDMSM(0, 1.0e8, 511000.0, 4.03)

Then we can calculate the rate by calling the function `rate(model, projected_gX, projected_fs2)`!

In [10]:
R = rate(model, proj_gx, proj_fs2)

1.2703889674263044e-11

Finally, rotations of the detector are important, so we can implement those as well. `rate` can also take a `Quaternion` or `Rotor` from the `Quaternionic.jl` package to describe a rotation. In this case, it becomes

`rate(rotor, model, projected_gX, projected_fs2)`

Note that `Quaternionic.jl` provides functions to convert to quaternions if you need them (e.g., Euler angles to quaternions).

In [11]:
using Quaternionic

In [12]:
rot = randn(RotorF64)
R_rotated = rate(rot, model, proj_gx, proj_fs2)

1.124275287766923e-11

`rate` will also accept a vector of quaternions or rotors, and providing this will calculate the rate at all rotations without taking hardly any more time than for a single rotation.

In [13]:
rots = randn(RotorF64, 10000)
Rs_rotated = rate(rots, model, proj_gx, proj_fs2)

10000-element Vector{Float64}:
 1.0201504161955817e-11
 1.0983226362514154e-11
 1.1285303397560842e-11
 9.874544000674672e-12
 1.0725736436732345e-11
 1.1890838001057508e-11
 1.0317279167907573e-11
 1.0368950858690714e-11
 1.203044528083029e-11
 1.0875514810650367e-11
 ⋮
 1.1227187617853651e-11
 1.1021032859500293e-11
 1.1600246256660578e-11
 1.165114096018712e-11
 9.940278025680462e-12
 1.1751566310602017e-11
 1.3461334499373984e-11
 1.0322977817326795e-11
 1.0735426238655662e-11

# Other useful things

One will probably want more than $n_{\textrm{max}} = 2^7 - 1$ and $\ell_{\textrm{max}} = 4$ coefficients when making calculations. Because it can often take a long time to run all of these coefficients, it's useful to be able to save and load them. You can easily do so with `writeFnlm` and `readFnlm`

```
    writeFnlm(outfile, pf::ProjectedF)

outfile : file to write coefficients to
```

and

```
    readFnlm(infile[, radial_basis])

infile : file to load
radial_basis : optional, specify basis ahead of time. Usually readFnlm will be able to load the basis from file
```